# Set-up

In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
papers = pd.read_excel('..\..\Table_1_Paper_metrics.xlsx')
datasets = pd.read_excel('..\..\Table_3_Dataset_Metrics.xlsx')

In [ ]:
# show columns
papers.columns

In [ ]:
datasets.columns

# Visualizations

In [ ]:
df = pd.read_excel('..\..\Table_1_Paper_metrics.xlsx')

In [ ]:
# Distribution of publication years
plt.figure()
min_year = 2006
max_year = int(df['Year'].max())
sns.histplot(data=df, x='Year', bins=(13), kde=False, linewidth=2, color='steelblue')
plt.xlim(min_year, max_year)
plt.xlabel('Publication Year')
plt.ylabel('Frequency')
plt.title('Distribution of Publication Years')

# save figure
plt.savefig('Distribution of Publication Years.png', dpi=300)

In [ ]:
# Number of datasets used for simulations
plt.figure(figsize=(16, 4))
ax = sns.boxplot(data=df, x='N_Datasets', linewidth=2, color='steelblue')
plt.xlabel('Number of Datasets')
plt.title('Distribution of Datasets Used for Simulations')
plt.xticks(range(0, 85, 5))

# Calculate the outliers using the interquartile range (IQR)
Q1 = df['N_Datasets'].quantile(0.25)
Q3 = df['N_Datasets'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify and annotate the outliers
outliers = df[(df['N_Datasets'] < lower_bound) | (df['N_Datasets'] > upper_bound)]
for index, row in outliers.iterrows():
    ax.annotate(int(row['N_Datasets']), (row['N_Datasets'], 0), textcoords="offset points", xytext=(-10,10), ha='center', fontsize=10, color='red')



# save figure
plt.savefig('Distribution of Datasets Used for Simulations.png', dpi=300)

# description
# this plot shows the distribution of the number of datasets used for simulations

In [ ]:
# Prelabeled datasets
plt.figure()
df['Labeled'].value_counts().plot(kind='pie', autopct='%1.1f%%', wedgeprops={'linewidth': 2, 'edgecolor': 'black'})
plt.title('Prelabeled Datasets')
plt.ylabel('')  # Hide the y-axis label

# save figure
plt.savefig('Prelabeled Datasets.png', dpi=300)

# a pie chart showing the percentage of papers that used prelabeled datasets

In [ ]:
# Split the metrics and count occurrences
metrics_counts = {}

for metrics in df['Metrics']:
    # Split by commas, then strip leading/trailing whitespaces and convert to lowercase
    split_metrics = [metric.strip().title() for metric in metrics.split(',')]

    for metric in split_metrics:
        if metric not in metrics_counts:
            metrics_counts[metric] = 1
        else:
            metrics_counts[metric] += 1

# Convert the metrics_counts dictionary to a Pandas Series for easier manipulation
metrics_counts_series = pd.Series(metrics_counts)

# combine recall and recall @ threshold and remove the old metrics
metrics_counts_series['Recall'] = metrics_counts_series['Recall'] + metrics_counts_series['Recall @ Threshold'] + metrics_counts_series['Recall @ Rank K'] + metrics_counts_series['Recall At Effort (Recall@E)']
metrics_counts_series = metrics_counts_series.drop(['Recall @ Threshold'])
metrics_counts_series = metrics_counts_series.drop(['Recall @ Rank K'])
metrics_counts_series = metrics_counts_series.drop(['Recall At Effort (Recall@E)'])

# lastrel% and last_rel
metrics_counts_series['Lastrel%'] = metrics_counts_series['Lastrel%'] + metrics_counts_series['Last_Rel'] + metrics_counts_series['Last Rel Found']
metrics_counts_series = metrics_counts_series.drop(['Last_Rel'])
metrics_counts_series = metrics_counts_series.drop(['Last Rel Found'])

# precision and precision @ last relevant found
metrics_counts_series['Precision'] = metrics_counts_series['Precision'] + metrics_counts_series['Precision @ Last Relevant Found']
metrics_counts_series = metrics_counts_series.drop(['Precision @ Last Relevant Found'])

# remove empty string
metrics_counts_series = metrics_counts_series.drop([''])

# wss and work saved
metrics_counts_series['Wss'] = metrics_counts_series['Wss'] + metrics_counts_series['Work Saved']
metrics_counts_series = metrics_counts_series.drop(['Work Saved'])

# Sort the metrics by count
sorted_metrics_counts = metrics_counts_series.sort_values(ascending=False)

In [ ]:
# barplot of metrics used in studies with at least 3 occurrences
plt.figure(figsize=(10, 6))
sns.barplot(x=sorted_metrics_counts[sorted_metrics_counts > 2].values, y=sorted_metrics_counts[sorted_metrics_counts > 2].index, orient='h', color='steelblue', edgecolor='black', linewidth=2)
plt.xlabel('Frequency')
plt.ylabel('Metrics')
plt.title('Frequency of Metrics Used in Studies')


# save figure
plt.savefig('Frequency of Metrics Used in Studies.png', dpi=300, bbox_inches='tight')
# a barplot showing the frequency of metrics used in studies

In [ ]:
# print all metrics 
for i in sorted_metrics_counts.index:
    print(i)

In [ ]:
# Split the data and count occurrences
data_counts = {}

for data in df['Data Used'].dropna():
    # Split by commas, then strip leading/trailing whitespaces and convert to lowercase
    split_data = [data.strip().lower() for data in data.split(',')]
    
    for data in split_data:
        if data not in data_counts:
            data_counts[data] = 1
        else:
            data_counts[data] += 1

# Convert the data_counts dictionary to a Pandas Series for easier manipulation
data_counts_series = pd.Series(data_counts)

# Sort the metrics by count
sorted_data_counts = data_counts_series.sort_values(ascending=False)

In [ ]:
# Barplot of data used in studies
plt.figure()
sorted_data_counts.plot(kind='barh', x='Data Type', y='Frequency', legend=None, width=0.75, edgecolor='black', linewidth=2, color='steelblue')
plt.xlabel('Frequency')
plt.ylabel('Data Type')
plt.gca().invert_yaxis()
plt.title('Types of Data Used to Train the Model')

# save figure
plt.savefig('Types of Data Used to Train the Model.png', dpi=300, bbox_inches='tight')

In [ ]:
# clean up OS_availability and OS_Reported
# change NaN to N/A in df['OS_availability'] 
df['OS_availability'] = df['OS_availability'].fillna('N/A')

# same for OS_Reported
df['OS_Reported'] = df['OS_Reported'].fillna('N/A')

# for OS_reported, combine all variants of dataset
df['OS_Reported'] = df['OS_Reported'].replace('Dataset ', 'Dataset')

# sane for OS_availability
df['OS_availability'] = df['OS_availability'].replace('Dataset ', 'Dataset')

In [ ]:
# Pie chart for OS_Reported
os_reported_counts = df['OS_Reported'].value_counts()
colors = {'Dataset': '#ffeda0', 'Dataset and code': '#a1d99b', 'N/A': '#fcbba1'}

plt.figure()
os_reported_counts.plot(kind='pie', autopct='%1.1f%%', colors=[colors[x] for x in os_reported_counts.index], wedgeprops={'linewidth': 2, 'edgecolor': 'black'})
plt.title('Data and Code Reported availability')
plt.ylabel('')  # Hide the y-axis label

# save figure
plt.savefig('OS_Reported.png', dpi=300)

In [ ]:
df['OS_availability'].value_counts()

In [ ]:
# Pie chart for OS_Reported
os_availability_counts = df['OS_availability'].value_counts()
colors = {'Dataset': '#ffeda0', 'Code': '#AEC6CF','Dataset and code': '#a1d99b', 'N/A': '#fcbba1'}

plt.figure()
os_reported_counts.plot(kind='pie', autopct='%1.1f%%', colors=[colors[x] for x in os_reported_counts.index], wedgeprops={'linewidth': 2, 'edgecolor': 'black'})
plt.title('Data and Code availability')
plt.ylabel('')  # Hide the y-axis label

# save figure
plt.savefig('OS_actual.png', dpi=300)

In [ ]:
# barplot for OS_availability
# Create a new DataFrame to store the actual availability information
availability_data = {
    'Dataset and code': {'True': 0, 'Dataset': 0, 'Code': 0, 'None': 0},
    'Dataset': {'Dataset': 0, 'None': 0},
    'N/A': {'None': 0}
}

for idx, row in df.iterrows():
    reported = row['OS_Reported']
    availability = row['OS_availability']
    if reported == 'Dataset and code':
        if availability == 'Dataset and code':
            availability_data[reported]['True'] += 1
        elif availability == 'Dataset':
            availability_data[reported]['Dataset'] += 1
        elif availability == 'Code':
            availability_data[reported]['Code'] += 1
        else:
            availability_data[reported]['None'] += 1
    elif reported == 'Dataset':
        if availability == 'Dataset':
            availability_data[reported]['Dataset'] += 1
        else:
            availability_data[reported]['None'] += 1
    else:
        if availability == 'N/A':
            availability_data[reported]['None'] += 1

availability_df = pd.DataFrame(availability_data).transpose()

# Plot the stacked bar chart
colors = ['#a1d99b', '#ffeda0', '#AEC6CF', '#fcbba1']
ax = availability_df.plot(kind='bar', stacked=True, color=colors, figsize=(8, 6),  width=0.75, edgecolor='black', linewidth=2)
plt.title('Actual Availability of Reported Resources')
plt.ylabel('Reported Frequency')
plt.xticks(rotation=0)

# Modify y-axis to show only integer values
ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))

# Add legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title='Actual Availability', loc='upper right')

# change label N/A to Not Available
ax.set_xticklabels(['Dataset and code reported', 'Dataset reported', 'None reported'])

# Change legend None to Not Available
ax.legend_.get_texts()[0].set_text('None Available')
ax.legend_.get_texts()[1].set_text('Only code')
ax.legend_.get_texts()[2].set_text('Only dataset')
ax.legend_.get_texts()[3].set_text('Available')

# save figure
plt.savefig('Actual Availability of Reported Resources.png', dpi=300)

# a barplot showing the actual availability of reported resources


In [ ]:
# calc what percentage of studies have data available
print('Percentage of studies with data available: {:.2f}%'.format(100 * (availability_df['True'].sum() + availability_df['Dataset'].sum()) / len(df)))